<a href="https://colab.research.google.com/github/noooah2000/learn-pytorch-assignments/blob/main/Custom_datasets_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 04. PyTorch Custom Datasets Exercises Template

In [ ]:
# Check for GPU
!nvidia-smi

In [ ]:
# Import torch
import torch
from torch import nn

# Exercises require PyTorch > 1.10.0
print(torch.__version__)

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

# Setup the random seed
RANDOM_SEED = 42

In [ ]:
# 取得資料
import requests
import zipfile
from pathlib import Path

data_path = Path('data')
image_path = data_path / 'pizza_steak_sushi'
zip_path = data_path / 'pizza_steak_sushi.zip'

if image_path.is_dir():
  print(f"{image_path} is exists")
else:
  print(f"Creating {image_path} now")
  image_path.mkdir(parents=True, exist_ok=True)

  with open(zip_path, 'wb') as f:
    request = requests.get('https://github.com/mrdbourke/pytorch-deep-learning/raw/refs/heads/main/data/pizza_steak_sushi.zip')
    print('Downloading the zip files of images ...')
    f.write(request.content)

  with zipfile.ZipFile(zip_path, 'r') as zipf:
    print('Unzipping the zip file ... ')
    zipf.extractall(image_path)


In [ ]:
# 設定資料路徑
train_dir = image_path / 'train'
test_dir = image_path / 'test'
print(train_dir)
print(test_dir)

In [ ]:
# 印出來看看
import random
from PIL import Image

random.seed(RANDOM_SEED)
image_path_list = list(image_path.rglob("*.jpg"))
random_image_path = random.choice(image_path_list)
image_class = random_image_path.parent.stem
img = Image.open(random_image_path)
print(f"Random image path: {random_image_path}")
print(f"Image class: {image_class}")
print(f"Image height: {img.height}")
print(f"Image width: {img.width}")
img


In [ ]:
# 用matplotlib印出來
import numpy as np
import matplotlib.pyplot as plt

img_array = np.asarray(img)
plt.figure(figsize=(10,7))
plt.imshow(img_array)
plt.axis(False)

In [ ]:
# Custom Dataset的helper function
import os

def FindClasses(directory):
  classes = sorted(entry.name for entry in os.scandir(path=directory) if entry.is_dir())
  if not classes:
    raise FileNotFoundError(f"Couldn't find any classes in {directory}.")
  class_to_idx = {class_name:idx for idx, class_name in enumerate(classes)}
  return classes, class_to_idx

FindClasses(test_dir)

In [ ]:
# 設計Custom Dataset(包含訓練資料的資料擴增設計)
from torch.utils.data import Dataset

class CustomDataset(Dataset):
  def __init__(self, target_dir, transform, transform_augment=None, augment=False):
    self.paths = list(target_dir.rglob('*.jpg'))
    self.transform = transform
    self.transform_augment = transform_augment
    self.augment = augment
    self.classes , self.class_to_idx = FindClasses(target_dir)

  def load_image(self, index):
    image_path = self.paths[index]
    return Image.open(image_path)

  def __len__(self):
    return 2*len(self.paths) if self.augment else len(self.paths)

  def __getitem__(self, index):
    if self.augment:
      original_index = index % len(self.paths)
      is_aug = (index >= len(self.paths))
    else:
      original_index = index
      is_aug = False

    class_name = self.paths[original_index].parent.name
    class_index = self.class_to_idx[class_name]
    img = self.load_image(original_index)

    if is_aug:
      return self.transform_augment(img), class_index
    else:
      return self.transform(img), class_index


In [ ]:
# 設定資料的transform處裡
import torchvision
from torchvision import transforms

train_transform = transforms.Compose([transforms.Resize((224, 224)), ## (224, 224)是由於transfer learning的默認size
                                      transforms.TrivialAugmentWide(num_magnitude_bins=31),
                                      transforms.ToTensor()])

transform_original = transforms.Compose([transforms.Resize((224, 224)),
                                         transforms.ToTensor()])

test_transform = transforms.Compose([transforms.Resize((224, 224)),
                                     transforms.ToTensor()])

In [ ]:
# 實體化Dataset
train_dataset = CustomDataset(target_dir=train_dir,
                              transform=transform_original,
                              transform_augment=train_transform,
                              augment=True)

test_dataset = CustomDataset(target_dir=test_dir,
                             transform=test_transform)

train_dataset_original = CustomDataset(target_dir=train_dir,
                                       transform=transform_original)
class_names = train_dataset.classes

test_dataset_labels = torch.tensor([test_dataset[i][1] for i in range(len(test_dataset))])
test_dataset_labels

In [ ]:
# 視覺化有無augmented的差別
random_idx_list = random.sample(range(len(train_dataset)), k=9)
plt.figure(figsize=(8,8))
for i, idx in enumerate(random_idx_list):
  image, label = train_dataset[idx]
  image = image.permute(1, 2, 0) #imshow()支援pytorch的tensor
  augment_status = idx // (len(train_dataset)/2)
  status = '(Augmented)' if augment_status else '(Unaugmented)'

  plt.subplot(3, 3, i+1)
  plt.imshow(image)
  plt.title(f"{class_names[label]}{status}\n{image.shape}", size=10, c='brown' if augment_status else 'black')
  plt.subplots_adjust(hspace=0.5, wspace=0.2)
  plt.axis(False)

In [ ]:
# 設定Dataloader
from torch.utils.data import DataLoader
BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()

train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=BATCH_SIZE,
                              num_workers=NUM_WORKERS,
                              shuffle=True)

test_dataloader = DataLoader(dataset=test_dataset,
                              batch_size=BATCH_SIZE,
                              num_workers=NUM_WORKERS,
                              shuffle=False)

next(iter(train_dataloader))[0].shape


In [ ]:
# 設定Transfer Learning的模型
torch.manual_seed(42)
torch.cuda.manual_seed(42)

weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
model = torchvision.models.efficientnet_b0(weights=weights).to(device)

for param in model.features.parameters():
    param.requires_grad = False

model.classifier = torch.nn.Sequential(torch.nn.Dropout(p=0.2, inplace=True),
                                       torch.nn.Linear(in_features=1280,
                                                       out_features=len(class_names), bias=True)
                                       ).to(device)

In [ ]:
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_func: torch.nn.Module,
               optimizer: torch.optim.Optimizer):
  model.train()
  train_loss, train_acc = 0, 0

  for X_train, y_train in tqdm(dataloader, desc='training', leave=False):
    X_train, y_train = X_train.to(device), y_train.to(device)

    y_logits = model(X_train)
    y_labels = torch.argmax(torch.softmax(y_logits, dim=1), dim=1)

    loss = loss_func(y_logits, y_train)
    train_loss += loss
    acc = (y_labels==y_train).sum().item() / len(y_labels)
    train_acc += acc

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

In [ ]:
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_func: torch.nn.Module):
  model.eval()
  test_loss, test_acc = 0, 0

  with torch.inference_mode():
    for X_test, y_test in tqdm(dataloader, desc='testing', leave=False):
      X_test, y_test = X_test.to(device), y_test.to(device)

      y_logits = model(X_test)
      y_labels = torch.argmax(torch.softmax(y_logits, dim=1), dim=1)

      loss = loss_func(y_logits, y_test)
      test_loss += loss
      acc = (y_labels==y_test).sum().item() / len(y_labels)
      test_acc += acc

  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc


In [ ]:
from tqdm.auto import tqdm

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_func: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5):
  results = {"train_loss": [],
             "train_acc": [],
             "test_loss": [],
             "test_acc": []}

  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model,
                                       dataloader=train_dataloader,
                                       loss_func=loss_func,
                                       optimizer=optimizer)
    test_loss, test_acc = test_step(model=model,
                                    dataloader=test_dataloader,
                                    loss_func=loss_func)

    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)

    train_acc, test_acc = train_acc*100, test_acc*100
    print(f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f}% | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}%"
    )

  return results

In [ ]:
# Train for 5 epochs
torch.manual_seed(42)
torch.cuda.manual_seed(42)

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(),
                             lr=0.001)

NUM_EPOCHS = 5
model_results = train(model=model,
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        optimizer=optimizer,
                        loss_func=loss_func,
                        epochs=NUM_EPOCHS)

In [ ]:
# 預測函數
def MakePredictions(model, some_sample_in_test_data, device):
  pred_label_list = []
  model.eval()
  with torch.inference_mode():
    for sample in some_sample_in_test_data:
      sample = torch.unsqueeze(sample, dim=0) #因為模型只吃(batch_size,C,H,W)的input，但sample沒有batch這個維度
      pred_logit = model(sample.to(device))
      pred_label = torch.argmax(pred_logit, dim=1)
      pred_label_list.append(pred_label.cpu())
  return torch.cat(pred_label_list) #把list合併成一個tensor

In [ ]:
import random
from random import sample, seed
#random.seed(42)
test_samples = []
test_labels = []
for sample, label in random.sample(list(test_dataset), k=16):
  test_samples.append(sample)
  test_labels.append(label)
test_samples[0].shape ,test_labels

In [ ]:
# 視覺化預測結果
pred_labels = MakePredictions(model=model,
                              some_sample_in_test_data=test_samples,
                              device=device)

plt.figure(figsize=(9, 9))
row, col = 4, 4
for idx, sample in enumerate(test_samples):
  plt.subplot(row, col, idx+1)
  plt.imshow(sample.permute(1, 2, 0).numpy())
  pred_label = pred_labels[idx]
  truth_label = test_labels[idx]
  title_text = f"Pred: {class_names[pred_label]} | Truth: {class_names[truth_label]}"
  plt.title(title_text, fontsize=10, c='g' if pred_label == truth_label else 'r')
  plt.axis(False)

In [ ]:
# 測試資料預測的結果
model.eval()
with torch.inference_mode():
  y_preds = torch.tensor([], dtype=torch.int32)
  for a_batch_of_X, _ in tqdm(test_dataloader, desc="Making predictions"):
    a_batch_of_y_preds = MakePredictions(model=model,
                            some_sample_in_test_data=a_batch_of_X,
                            device=device)

    y_preds = torch.cat((y_preds, a_batch_of_y_preds), dim=0)
print(y_preds[:100])

In [ ]:
try:
  import torchmetrics, mlxtend
except:
  !pip install -q torchmetrics -U mlxtend
  import torchmetrics, mlxtend
print(f"mlxtend version: {mlxtend.__version__}")

In [ ]:
# 用Confusion Matrix分析模型精準度
from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix

confmat = ConfusionMatrix(num_classes=len(class_names),
                          task='multiclass')
confmat_tensor = confmat(preds=y_preds,
                         target=test_dataset_labels)
plot_confusion_matrix(conf_mat=confmat_tensor.numpy(),
                      class_names=class_names,
                      figsize=(10, 7))
for idx in range(len(class_names)):
  count = torch.sum((y_preds == idx)&(test_dataset_labels == idx))
  print(f"Count of correct predictions for class [{class_names[idx]}]: {count}")